<a href="https://colab.research.google.com/github/lilasch/Facial_Attribute_Classification/blob/main/liladatapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Facial Attribute Classification**

Lila Schisgal and Phoebe Jeske


First, we import our data from two text files and partition it into training and validation data.

In [ ]:
import tensorflow.keras as keras
import pandas as pd
import cv2
#import autotrace
import numpy as np
import glob


from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# 40 attributes in total
num_attributes = 40

# Load attributes csv
attributes = pd.read_csv("/content/drive/MyDrive/list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")

# Load csv with partitions values
partitions = pd.read_csv("/content/drive/MyDrive/list_eval_partition.txt", delimiter="\s+|\t", header = None)

partitions.columns = ['image_name', 'dataset'] # setting column header names for partitions
attributes['dataset'] = partitions['dataset'].values # copying the partition values into the attributes df

# filter so just the eyeglasses attribute is included
glasses_df = attributes.iloc[:, [15, num_attributes]]

# partitioning the data
train_df = glasses_df[glasses_df['dataset'] == 0]
valid_df = glasses_df[glasses_df['dataset'] == 1]


Mounted at /content/drive


<ipython-input-16-e8088da7fb98>:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  attributes = pd.read_csv("/content/drive/MyDrive/list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")
<ipython-input-16-e8088da7fb98>:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  partitions = pd.read_csv("/content/drive/MyDrive/list_eval_partition.txt", delimiter="\s+|\t", header = None)


In [ ]:
print(train_df)

            Eyeglasses  dataset
000001.jpg          -1        0
000002.jpg          -1        0
000003.jpg          -1        0
000004.jpg          -1        0
000005.jpg          -1        0
...                ...      ...
162766.jpg          -1        0
162767.jpg          -1        0
162768.jpg          -1        0
162769.jpg          -1        0
162770.jpg          -1        0

[162770 rows x 2 columns]


We then separate our target y values and image filenames in preparation for accessing the actual images as vectors.

In [ ]:
# separating out target values
y_train = train_df['Eyeglasses']
y_valid = valid_df['Eyeglasses']
del train_df['Eyeglasses']
del valid_df['Eyeglasses']

# getting our image file names in two arrays
train_files = train_df.index
valid_files = valid_df.index

# Some visualization of these datasets
# attributes.head()
# glasses_df.head()
# print(train_files[0])

In [ ]:
# unzipping image data
import zipfile
zippath = '/content/sample_data/Celeba.zip'
targetfolder = '/content/sample_data'

with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(targetfolder)

BadZipFile: File is not a zip file

Next: need to access the images from the dataset and put them into a set of vectors.

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Set your folder paths
input_folder = "/content/drive/MyDrive/images_celebA"
output_folder = "/content/drive/MyDrive/preprocessed_images"

# Function to normalize images
def normalize_images(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            print(img_path)
            img = Image.open(img_path)
            img_array = np.array(img)
            print(img_array.shape)
            flattened_dim = 218 * 178 * 3
            img_processed = img_array.reshape(-1, flattened_dim)
            img_processed = img_processed.astype(float)

def standardize(train, test, valid):
    mean = np.mean(train)
    std = np.std(train)

    train = (train - mean)/std
    valid = (valid - mean)/std
    test  = (test - mean)/std

    return train, valid, test

# Apply preprocessing steps
normalize_images(input_folder)


/content/drive/MyDrive/images_celebA/000002.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000008.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000010.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000001.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000004.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000009.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000003.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000007.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000005.jpg
(218, 178, 3)
/content/drive/MyDrive/images_celebA/000006.jpg
(218, 178, 3)
